In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv', index_col=[0])
test = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')

In [4]:
train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [5]:
test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148


In [6]:
train['Gender'].nunique(), test['Gender'].nunique()

(2, 2)

In [9]:
def label_encode_gender(val):
    if val == 'Female':
        return 1
    elif val == 'Male':
        return 0
    return None

In [10]:
train['Gender'] = train['Gender'].apply(label_encode_gender)
test['Gender'] = test['Gender'].apply(label_encode_gender)

In [11]:
train['Vehicle_Age'].value_counts(), test['Vehicle_Age'].value_counts()

(Vehicle_Age
 1-2 Year     5982678
 < 1 Year     5044145
 > 2 Years     477975
 Name: count, dtype: int64,
 Vehicle_Age
 1-2 Year     3989276
 < 1 Year     3361255
 > 2 Years     319335
 Name: count, dtype: int64)

In [16]:
def label_encode_vehicle_age(val):
    if val == '< 1 Year':
        return 0
    elif val == '1-2 Year':
        return 1
    elif val == '> 2 Years':
        return 2
    return None

In [19]:
train['Vehicle_Age'] = train['Vehicle_Age'].apply(label_encode_vehicle_age)
test['Vehicle_Age'] = test['Vehicle_Age'].apply(label_encode_vehicle_age)

In [20]:
train['Vehicle_Damage'].unique()

array(['Yes', 'No'], dtype=object)

In [21]:
def label_encode_vehicle_damage(val):
    if val == 'Yes':
        return 1
    else:
        return 0
    return None

In [23]:
train['Vehicle_Damage'] = train['Vehicle_Damage'].apply(label_encode_vehicle_damage)
test['Vehicle_Damage'] = test['Vehicle_Damage'].apply(label_encode_vehicle_damage)

In [24]:
train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,,,,,,,,,,,
0,0,21,1,35.0,0,1,1,65101.0,124.0,187,0
1,0,43,1,28.0,0,2,1,58911.0,26.0,288,1
2,1,25,1,14.0,1,0,0,38043.0,152.0,254,0
3,1,35,1,1.0,0,1,1,2630.0,156.0,76,0
4,1,36,1,15.0,1,1,0,31951.0,152.0,294,0


In [25]:
test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,1,20,1,47.0,0,0,0,2630.0,160.0,228
1,11504799,0,47,1,28.0,0,1,1,37483.0,124.0,123
2,11504800,0,47,1,43.0,0,1,1,2630.0,26.0,271
3,11504801,1,22,1,47.0,1,0,0,24502.0,152.0,115
4,11504802,0,51,1,19.0,0,1,0,34115.0,124.0,148


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11504798 entries, 0 to 11504797
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Gender                int64  
 1   Age                   int64  
 2   Driving_License       int64  
 3   Region_Code           float64
 4   Previously_Insured    int64  
 5   Vehicle_Age           int64  
 6   Vehicle_Damage        int64  
 7   Annual_Premium        float64
 8   Policy_Sales_Channel  float64
 9   Vintage               int64  
 10  Response              int64  
dtypes: float64(3), int64(8)
memory usage: 1.0 GB


In [29]:
y = train['Response']
X = train.drop(columns=['Response'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
y_pred_train_proba = model.predict_proba(X_test)[:, 1]
y_pred_test_proba = model.predict_proba(X_test)[:, 1]

print(f'Train AUC = {roc_auc_score(y_train, y_pred_train_proba)}')
print(f'Test AUC = {roc_auc_score(y_test, y_pred_test_proba)}')